In [1]:
%load_ext nb_mypy

Version 1.0.5


In [2]:
from loguru import logger
from s3path import S3Path
from datetime import datetime

from sm_pipelines_oo.config_loader.implementations.file_loaders import YamlConfigLoader
from sm_pipelines_oo.pipeline import PipelineFacade, DevPipelineFacade
from sm_pipelines_oo.shared_config_schema import Environment

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/thomas-22/.config/sagemaker/config.yaml


In [3]:
_PROJECT_BUCKET = S3Path('/smp-oo-example')
_DATA_PREFIX = 'examples/data/'
output_filename = f'output-{datetime.now()}'

ENV: Environment = 'dev'
_config_loader = YamlConfigLoader(
    env=ENV,
    config_root_folder='../../config'  # relative to package root
)

In [4]:
# Create input dataframe (with current date for validation) and save it to S3
from smp_oo_examples.validation import setup_input

input_path_s3 = f"{_config_loader.step_configs_as_dicts[0]['processor_run_config']['inputs']['input_3']}/input.parquet"
validation_data = setup_input(input_path_s3=input_path_s3)

INFO:botocore.tokens:Loading cached SSO token for ml


In [5]:
pipeline = DevPipelineFacade(
    env=ENV,
    # Use different configs for testing
    custom_config_loader=_config_loader,
)


INFO:botocore.tokens:Loading cached SSO token for ml
/home/thomas-22/.cache/pypoetry/virtualenvs/sm-pipelines-oo-examples-yainlQgE-py3.10/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [6]:
pipeline.create_and_start_pipeline_from_definition()

INFO:botocore.tokens:Loading cached SSO token for ml
INFO:sagemaker.processing:Uploaded worker_code/preprocess to s3://sagemaker-us-east-1-338755209567/test-v0-1/code/3b53fec93037e15018c750ec711dfc75/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-338755209567/test-v0-1/code/bc2536a25d34e1ecae5238f42f4207c2/runproc.sh
/home/thomas-22/.cache/pypoetry/virtualenvs/sm-pipelines-oo-examples-yainlQgE-py3.10/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(
2024-03-22 09:26:06.438 | INFO     | sm_pipelines_oo.pipeline:export_pipeline_definition_to_s3:82 - Uploaded pipeline definition to s3://smp-oo-example/pipeline_definitions/test-v0-1.json

An error occurred (ValidationException) when calling the CreatePipeline operation: Pipeline names must be unique within an AWS account and region. Pipeline with name (t

In [10]:
from smp_oo_examples.validation import check_output

_output_s3_dir = _config_loader.step_configs_as_dicts[0]['processor_run_config']['outputs']['output_1']
_output_filename = _config_loader.step_configs_as_dicts[0]['processor_init_config']['env']['OUTPUT_FILENAME']
check_output(
    wait_time_in_minutes=15,
    df_in= validation_data['input'],
    output_path_s3=f'{_output_s3_dir}/{_output_filename}',
)

   a  b                             date
0  2  4 2024-03-22 13:30:19.021954+00:00
1  2  4 2024-03-22 13:30:19.021954+00:00

Observed timedelta: 0 days 00:12:02.586698


In [11]:
print(datetime.now())

2024-03-22 09:42:22.526037
